In [ ]:
# 使用结巴分词对

In [39]:
# 加载评测数据集包含了

from datasets import load_dataset
import os
from datasets import Dataset
dataset_path="../data/rag_data/eval/eval_dataset2"
if os.path.isdir(dataset_path):
    dataset=Dataset.load_from_disk(dataset_path)

# 因为要调用文心评测 这里只用数据集的一个样本
dataset_mini=dataset.select(range(25))

In [ ]:
# 以使用ERNIE-Bot为，使用qianfan-sdk先定义自己的模型
from langchain_community.chat_models import QianfanChatEndpoint
from ragas.llms import LangchainLLMWrapper
from langchain_community.embeddings import QianfanEmbeddingsEndpoint
import os
import qianfan
import pandas as pd

QIANFAN_ACCESS_KEY="eK8QT20CEUvhggAoO54HaSqI"
QIANFAN_SECRET_KEY="AMEbJ2sLJikNvSLublIikzGpBnig10py"
chat = QianfanChatEndpoint(model="ERNIE-3.5-8K", qianfan_ak=QIANFAN_ACCESS_KEY, qianfan_sk=QIANFAN_SECRET_KEY,request_timeout=120)
vllm = LangchainLLMWrapper(chat)

v_embeddings = QianfanEmbeddingsEndpoint(
               qianfan_ak=QIANFAN_ACCESS_KEY,
               qianfan_sk=QIANFAN_SECRET_KEY,
           )

# 然后重新指定各评价指标使用的llm
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

faithfulness.llm = vllm
answer_relevancy.llm = vllm
answer_relevancy.embeddings = v_embeddings

context_recall.llm = vllm
context_precision.llm = vllm

# 重新一键式测评
result = evaluate(
    dataset_mini,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

df = result.to_pandas()
df.head()
df.to_csv('../data/rag_data/eval/evaleval_result.csv', index=False,encoding="utf-8")

In [ ]:
df=pd.read_csv('../data/rag_data/eval/evaleval_result.csv')
df

In [176]:
## 使用jieba分词后 通过ntlk计算bleu  和  计算rouge  其中ronge是字典 里面有 1，2，l  里面还有r p l

from nltk.translate.bleu_score import sentence_bleu 
from nltk.translate.bleu_score import SmoothingFunction
from rouge_chinese import Rouge
import jieba # you can use any other word cutting library

def compute_rouge_blue(answer_list,ground_truth_list):

    rouge = Rouge()
    chencherry = SmoothingFunction()

    rouge_score=[]
    bleu_score=[]
    for i in zip(answer_list,ground_truth_list):
        hypothesis=i[0]
        hypothesis_iter=jieba.cut(hypothesis)
        hypothesis = ' '.join(hypothesis_iter) 

        hypothesis_iter=jieba.cut(hypothesis) # 使用一次要重新cut
        hypothesis_bleu_format=list(hypothesis_iter)


        reference=i[1]
        reference_iter=jieba.cut(reference)
        reference = ' '.join(reference_iter)

        reference_iter=jieba.cut(reference)
        reference_bleu_format=list(reference_iter)


        rouge_score.append(rouge.get_scores(hypothesis, reference))
        bleu_score.append(sentence_bleu([reference_bleu_format], hypothesis_bleu_format,smoothing_function=chencherry.method1) )
    
    return  rouge_score,bleu_score
rouge_score,bleu_score=compute_rouge_blue(dataset_mini['answer'],dataset_mini['ground_truth'])

In [177]:
rouge_1_r,rouge_1_p,rouge_1_f=[],[],[]

rouge_2_r,rouge_2_p,rouge_2_f=[],[],[]

rouge_l_r,rouge_l_p,rouge_l_f=[],[],[]

for i in rouge_score:

    rouge_1_r.append(i[0]['rouge-1']['r'])
    rouge_1_p.append(i[0]['rouge-1']['p'])
    rouge_1_f.append(i[0]['rouge-1']['f'])

    rouge_2_r.append(i[0]['rouge-2']['r'])
    rouge_2_p.append(i[0]['rouge-2']['p'])
    rouge_2_f.append(i[0]['rouge-2']['f'])

    rouge_l_r.append(i[0]['rouge-l']['r'])
    rouge_l_p.append(i[0]['rouge-l']['p'])
    rouge_l_f.append(i[0]['rouge-l']['f'])

df['rag_rouge_1_r']=rouge_1_r
df['rag_rouge_1_p']=rouge_1_p
df['rag_rouge_1_f']=rouge_1_f

df['rag_rouge_2_r']=rouge_2_r
df['rag_rouge_2_p']=rouge_2_p
df['rag_rouge_2_f']=rouge_2_f

df['rag_rouge_l_r']=rouge_l_r
df['rag_rouge_l_p']=rouge_l_p
df['rag_rouge_l_f']=rouge_l_f

df['rag_bleu_score']=bleu_score

In [ ]:
import json
llm_answers_path="../data/rag_data/eval/llm_answers.json"

with open(llm_answers_path,"r",encoding='utf-8') as file:
    llm_answers_json=json.load(file)
    
rouge_score,bleu_score=compute_rouge_blue([item['llm_answers'] for item in llm_answers_json],dataset_mini['ground_truth'])
rouge_score,bleu_score


In [179]:
rouge_1_r,rouge_1_p,rouge_1_f=[],[],[]

rouge_2_r,rouge_2_p,rouge_2_f=[],[],[]

rouge_l_r,rouge_l_p,rouge_l_f=[],[],[]

for i in rouge_score:

    rouge_1_r.append(i[0]['rouge-1']['r'])
    rouge_1_p.append(i[0]['rouge-1']['p'])
    rouge_1_f.append(i[0]['rouge-1']['f'])

    rouge_2_r.append(i[0]['rouge-2']['r'])
    rouge_2_p.append(i[0]['rouge-2']['p'])
    rouge_2_f.append(i[0]['rouge-2']['f'])

    rouge_l_r.append(i[0]['rouge-l']['r'])
    rouge_l_p.append(i[0]['rouge-l']['p'])
    rouge_l_f.append(i[0]['rouge-l']['f'])

df['llm_rouge_1_r']=rouge_1_r
df['llm_rouge_1_p']=rouge_1_p
df['llm_rouge_1_f']=rouge_1_f

df['llm_rouge_2_r']=rouge_2_r
df['llm_rouge_2_p']=rouge_2_p
df['llm_rouge_2_f']=rouge_2_f

df['llm_rouge_l_r']=rouge_l_r
df['llm_rouge_l_p']=rouge_l_p
df['llm_rouge_l_f']=rouge_l_f

df['llm_bleu_score']=bleu_score

In [180]:
column_names = ['context_precision', 'faithfulness', 'answer_relevancy', 'context_recall','rag_rouge_1_r','rag_rouge_1_p','rag_rouge_1_f','rag_rouge_2_r','rag_rouge_2_p','rag_rouge_2_f','rag_rouge_l_r','rag_rouge_l_p','rag_rouge_l_f','rag_bleu_score',
                'llm_rouge_1_r','llm_rouge_1_p','llm_rouge_1_f','llm_rouge_2_r','llm_rouge_2_p','llm_rouge_2_f','llm_rouge_l_r','llm_rouge_l_p','llm_rouge_l_f','llm_bleu_score'
                ]
print(df[column_names].mean())
df.to_csv('../data/rag_data/eval/evaleval_result_all.csv', index=False,encoding="utf-8")

context_precision    0.840000
faithfulness         0.915686
answer_relevancy     0.776482
context_recall       0.766000
rag_rouge_1_r        0.407527
rag_rouge_1_p        0.429279
rag_rouge_1_f        0.383734
rag_rouge_2_r        0.204050
rag_rouge_2_p        0.218466
rag_rouge_2_f        0.187506
rag_rouge_l_r        0.322901
rag_rouge_l_p        0.309480
rag_rouge_l_f        0.272518
rag_bleu_score       0.218051
llm_rouge_1_r        0.381247
llm_rouge_1_p        0.293713
llm_rouge_1_f        0.306994
llm_rouge_2_r        0.141714
llm_rouge_2_p        0.100079
llm_rouge_2_f        0.105389
llm_rouge_l_r        0.300244
llm_rouge_l_p        0.188993
llm_rouge_l_f        0.198273
llm_bleu_score       0.163098
dtype: float64


In [182]:
sum([df[column_names].mean()[i] for i in ['rag_rouge_1_r','rag_rouge_1_p','rag_rouge_1_f','rag_rouge_2_r','rag_rouge_2_p','rag_rouge_2_f','rag_rouge_l_r','rag_rouge_l_p','rag_rouge_l_f','rag_bleu_score']])/10,\
sum([df[column_names].mean()[i] for i in ['llm_rouge_1_r','llm_rouge_1_p','llm_rouge_1_f','llm_rouge_2_r','llm_rouge_2_p','llm_rouge_2_f','llm_rouge_l_r','llm_rouge_l_p','llm_rouge_l_f','llm_bleu_score']])/10


(0.2953512089618152, 0.21797452247345261)